### Aug

In [1]:
import pandas as pd
import json
from collections import defaultdict as ddict
from datetime import date
import lexical_features as lf
import brandseg
from tqdm.notebook import tqdm

bs = brandseg.BrandSeg()

In [2]:
domains_vt1 = '/bad_ip_data/ucsc/Vaccine_Disinfo/data/covid_resources/dataset_aug_2020/oxford-vaccine-article_urls-shared-on-twitter-17-08-to-26-08.csv.1.scan.report'
domains_vt2 = '/bad_ip_data/ucsc/Vaccine_Disinfo/data/covid_resources/dataset_aug_2020/oxford-vaccine-article_urls-shared-on-twitter-17-08-to-26-08.csv.2.scan.report'

In [3]:
domains_vt = [i.split('\n')[0] for i in open(domains_vt1,'r').readlines()]+[i.split('\n')[0] for i in open(domains_vt2,'r').readlines()]

In [4]:
def timeduration(d0,d1):
    d0y,d0m,d0d =  list(map(int,d0.split(' ')[0].split('-')))
    d1y,d1m,d1d =  list(map(int,d1.split(' ')[0].split('-')))
    return (date(d1y,d1m,d1d)-date(d0y,d0m,d0d)).days

In [5]:
import tldextract
VT = ddict(list)
for i in tqdm(domains_vt):
    try:
        tmp = json.loads(i)
        url = tldextract.extract(tmp.get('url'))
        VT['domain'].append('{}.{}'.format(url.domain,url.suffix))
        VT['scanned url'].append(tmp.get('url'))
        VT['total'].append(tmp.get('total'))
        VT['positives'].append(tmp.get('positives'))
        VT['duration[VT]'].append(timeduration(tmp.get('first_seen'),tmp.get('last_seen')))
        
        URL_FINAL = tmp.get('url')
        
        try:
            rlength = tmp.get('additional_info').get('rlength')
            if rlength != None : VT['rlength'].append(rlength)
            if rlength == None : VT['rlength'].append(None)
        except AttributeError:
            VT['rlength'].append(None)
        
        try:
            resolution = tmp.get('additional_info').get('resolution')
            if resolution != None : VT['resolution'].append(resolution)
            if resolution == None : VT['resolution'].append(None)
        except AttributeError:
            VT['resolution'].append(None)
            
        try:    
            server = tmp.get('additional_info').get('Response headers').get('server')
            if server != None: VT['server'].append(server)
            if server == None: VT['server'].append(None)
        except AttributeError:
            VT['server'].append(None)
        
        try:
            URLAfterRredirects = tmp.get('additional_info').get('URL after redirects')
            if URLAfterRredirects != None: 
                VT['URL after redirects'].append(URLAfterRredirects)
                VT['Final URL'].append(URLAfterRredirects)
                URL_FINAL = URLAfterRredirects
            if URLAfterRredirects == None:
                VT['URL after redirects'].append('None')
                VT['Final URL'].append(tmp.get('url'))
        except AttributeError:
            VT['URL after redirects'].append(None)
            VT['Final URL'].append(tmp.get('url'))
            
        lex = lf.get_features(URL_FINAL, bs)
            
        VT['suspicious_tld'].append(lex.get('suspicious_tld'))
        VT['length'].append(lex.get('length'))
        VT['fake_tld'].append(lex.get('fake_tld'))
        VT['brand'].append(lex.get('brand'))
        VT['pop_keywords'].append(lex.get('pop_keywords'))
        VT['similar'].append(lex.get('similar'))
        VT['is_idn'].append(lex.get('is_idn'))
        VT['minus'].append(lex.get('minus'))
        VT['num_subdomains'].append(lex.get('num_subdomains'))
            
    except:
        pass
VT = pd.DataFrame(VT)

In [6]:
VT

,domain,scanned url,total,positives,duration[VT],rlength,resolution,server,URL after redirects,Final URL,suspicious_tld,length,fake_tld,brand,pop_keywords,similar,is_idn,minus,num_subdomains
0,reddit.com,https://www.reddit.com/r/u_rmaccr/comments/ibi...,83,0,0,62335.0,151.101.65.140,snooserv,https://www.reddit.com/user/rmaccr/comments/ib...,https://www.reddit.com/user/rmaccr/comments/ib...,N,188,0,1,0,1,N,0,1
1,sputniknews.com,https://fr.sputniknews.com/international/20200...,83,0,0,112305.0,178.248.237.18,QRATOR,None,https://fr.sputniknews.com/international/20200...,N,152,0,0,0,0,N,16,1
2,minci.gob.ve,http://www.minci.gob.ve/primer-lote-de-la-vacu...,83,0,0,73169.0,201.249.71.50,Apache/2.4.10 (Debian),None,http://www.minci.gob.ve/primer-lote-de-la-vacu...,N,89,0,0,0,0,N,12,2
3,sptnkne.ws,https://sptnkne.ws/Dv3G,83,0,0,87110.0,195.93.247.47,QRATOR,https://tr.sputniknews.com/turkiye/20200817104...,https://tr.sputniknews.com/turkiye/20200817104...,N,105,0,0,0,0,N,6,1
4,hispanosaldia.com,http://hispanosaldia.com/2020/08/17/inmunologa...,83,0,0,118769.0,172.67.144.76,cloudflare,None,http://hispanosaldia.com/2020/08/17/inmunologa...,N,117,0,0,0,0,N,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13663,sputnik.kg,https://ru.sputnik.kg/economy/20200826/1049426...,83,0,0,85043.0,176.126.167.188,None,None,https://ru.sputnik.kg/economy/20200826/1049426...,N,108,0,0,0,0,N,4,2
13664,sputniknews.com,https://rs.sputniknews.com/vesti/2020082611232...,83,0,0,93042.0,178.248.237.242,QRATOR,None,https://rs.sputniknews.com/vesti/2020082611232...,N,199,0,0,0,0,N,10,1
13665,sptnkne.ws,https://sptnkne.ws/D2p3,83,0,0,105677.0,195.93.247.47,QRATOR,https://fr.sputniknews.com/france/202008261044...,https://fr.sputniknews.com/france/202008261044...,N,133,0,0,0,0,N,13,1
13666,twitter.com,https://twitter.com/AGHamilton29/status/129850...,83,0,0,2251.0,104.244.42.65,tsa_b,https://mobile.twitter.com/AGHamilton29/status...,https://mobile.twitter.com/AGHamilton29/status...,N,66,0,1,0,1,N,0,1


In [7]:
VT.to_csv('/bad_ip_data/ucsc/Vaccine_Disinfo/data/FULLDF/aug_cluster.csv')